# GHG Emissions Implementation

## Imports

In [ ]:
import os
import pandas as pd
import numpy as np

## Global Variables

In [ ]:
COLAB = True

In [ ]:
ROOT_DIR_PATH = os.path.abspath('..')

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

  ROOT_DIR_PATH = os.path.abspath('drive/MyDrive/Spatial_Finance_Transport/')

ROOT_VEHICLE_DETECTION_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/vehicle_counts/vehicle_counts_')
EMISSIONS_FACTORS_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/uk_emissions_factors.csv')
VEHICLE_COUNTS_ROOT_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/vehicle_counts/')

COUNT_SITE_PATHS = ['luton_m1_2557A.csv', 'luton_m1_2557B.csv', 'havering_m25_5790A.csv', 'havering_m25_5790B.csv', 
                           'hounslow_m4_2188A.csv', 'hounslow_m4_2188B.csv',
                           'blackburn_30361033.csv', 'blackburn_30361032.csv']

AADT_ROOT_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/aadt/')
GHG_EMISSIONS_ROOT_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/ghg_emissions/')

Mounted at /content/drive


In [ ]:
VEHICLE_CATEGORIES = ['Passenger Vehicle',
  'Small Car',
  'Bus',
  'Pickup Truck',
  'Utility Truck',
  'Truck',
  'Cargo Truck',
  'Truck w/Box',
  'Truck Tractor',
  'Trailer',
  'Truck w/Flatbed',
  'Truck w/Liquid',
  'Passenger Car'
]

EMISSIONS_CATEGORY_MAPPING = {
    'Passenger Vehicle': 'Petrol cars',
    'Small Car': 'Petrol cars',
    'Pickup Truck': 'Petrol LGVs',
    'Utility Truck': 'Petrol LGVs',
    'Truck': 'Petrol LGVs',
    'Cargo Truck': 'Rigid HGVs',
    'Truck Tractor': 'Rigid HGVs',
    'Trailer': 'Petrol LGVs',
    'Truck w/Flatbed': 'Rigid HGVs',
    'Truck w/Liquid': 'Rigid HGVs',
    'Passenger Car': 'Petrol cars',
    'Truck w/Box': 'Petrol LGVs',
    'Bus': 'Buses',
    'Trailer': 'Petrol LGVs',
    'Cargo Car': 'Petrol LGVs'
}

XVIEW_TO_UK_MAPPING = {
    'Passenger Vehicle': 'cars_and_taxis',
    'Small Car': 'cars_and_taxis',
    'Pickup Truck': 'lgvs',
    'Utility Truck': 'lgvs',
    'Truck': 'lgvs',
    'Cargo Truck': 'all_hgvs',
    'Truck Tractor': 'all_hgvs',
    'Trailer': 'lgvs',
    'Truck w/Flatbed': 'all_hgvs',
    'Truck w/Liquid': 'all_hgvs',
    'Passenger Car': 'cars_and_taxis',
    'Truck w/Box': 'lgvs',
    'Bus': 'buses_and_coaches',
    'Trailer': 'lgvs',
    'Cargo Car': 'lgvs'
}


VEHICLE_KM_PER_LITRE_MAPPING = {
    'aadt': 15,
    'cars_and_taxis': 20,
    'buses_and_coaches': 3,
    'lgvs': 15,
    'all_hgvs': 3.6
}

KG_TO_KT = 1e-6

# Total kg CO2e per unit litres
PETROL = 2.16
DIESEL = 2.56

PETROL_DIESEL_AVERAGE = 0.36 * DIESEL + 0.64 * PETROL

In [ ]:
# miles
LUTON_ROAD_LENGTH = 2.6
BLACKBURN_ROAD_LENGTH = 8.0
HOUNSLOW_ROAD_LENGTH = 9.8
HAVERING_ROAD_LENGTH = 11.8
TRAFFORD_ROAD_LENGTH = 6.2

In [ ]:
# km
LUTON_ROAD_LENGTH = 4.18
BLACKBURN_ROAD_LENGTH = 12.87
HOUNSLOW_ROAD_LENGTH = 15.77
HAVERING_ROAD_LENGTH = 19
TRAFFORD_ROAD_LENGTH = 9.98

## Helper Functions

In [ ]:
def save_float_to_csv(float_value, column_name, image_id, file_name):
    """
    Save a float value to a CSV file with the specified column name and file name.
    
    Args:
        float_value (float): The float value to be saved.
        column_name (str): The name of the column in the CSV file.
        file_name (str): The name of the CSV file to be saved.
    """
    # Create a DataFrame with a single row and the specified column name and value
    df = pd.DataFrame({'image_id': image_id, column_name: [float_value]})
    
    # Save the DataFrame to a CSV file
    df.to_csv(file_name, index=False)

In [ ]:
def get_files_in_directory(directory):
    """
    Get a list of all files in a directory.

    Args:
        directory (str): Directory path.

    Returns:
        list: List of files in the directory.
    """
    files = []
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            files.append(file_path)
    return files

In [ ]:
def convert_category_names(dataframes_list, mapping_dict):
    """
    Convert category names in a list of DataFrames using a mapping dictionary.

    Args:
        dataframes_list (list): List of DataFrames with 'category_name' column.
        mapping_dict (dict): Dictionary containing mapping of old category names to new category names.

    Returns:
        list: List of DataFrames with updated category names.
    """
    updated_dataframes = []
    for df in dataframes_list:
        df['Vehicle Type'] = df['category_name'].map(mapping_dict)
        updated_dataframes.append(df)
    return updated_dataframes

In [ ]:
def calculate_ghg_emissions(car_category, emission_factors, road_length):
    """
    Calculates GHG emissions for a given car category, emission factors, and road length.
    
    Args:
        car_category (str): Category of the car (e.g., "Small Car", "Midsize Car", etc.).
        emission_factors (dict): Dictionary containing emission factors for different car categories.
        road_length (float): Length of the road segment in kilometers.
        
    Returns:
        float: Total GHG emissions in kilograms for the given car category and road length.
    """
    # Check if the emission factors dictionary contains the given car category
    if car_category not in emission_factors:
        raise ValueError("Car category not found in emission factors dictionary.")
    
    # Get the emission factors for the given car category
    car_emission_factors = emission_factors[car_category]
    
    # Calculate GHG emissions using the emission factors and road length
    ghg_emissions = car_emission_factors['co2'] * road_length + \
                    car_emission_factors['ch4'] * road_length + \
                    car_emission_factors['n2o'] * road_length
    
    return ghg_emissions

In [ ]:
def add_total_column(dataframes_list, other_dataframe):
    """
    Add a 'Total' column from one DataFrame to each DataFrame in a list of DataFrames based on the 'Vehicle Type' column.

    Args:
        dataframes_list (list): List of DataFrames.
        other_dataframe (DataFrame): DataFrame to extract the 'Total' column from.

    Returns:
        list: List of DataFrames with the 'Total' column added.
    """
    updated_dataframes = []
    for df in dataframes_list:
        if 'Vehicle Type' in df.columns and 'Vehicle Type' in other_dataframe.columns:
            total_column = other_dataframe[['Vehicle Type', 'Total']]
            df = df.merge(total_column, on='Vehicle Type', how='left')
        updated_dataframes.append(df)
    return updated_dataframes


In [ ]:
def create_vehicle_type_counts_df(df):
    """
    Count unique vehicle types in a DataFrame and return a DataFrame with columns as vehicle types and
    a single row with counts as values.
    
    Args:
        df (pandas.DataFrame): DataFrame containing the columns: image_id, x_min, x_max, y_min, y_max,
                               category_name, area, Vehicle Type, and Total.
                               
    Returns:
        pandas.DataFrame: DataFrame with columns as vehicle types and a single row with counts as values.
    """
    # Check if "Vehicle Type" column is present in the DataFrame
    if "Vehicle Type" not in df.columns:
        raise ValueError("Column 'Vehicle Type' not found in the DataFrame.")
    
    # Count unique values in "Vehicle Type" column
    vehicle_type_counts = df["Vehicle Type"].value_counts().to_dict()
    
    # Create a DataFrame from the counts dictionary
    counts_df = pd.DataFrame(vehicle_type_counts, index=[0])
    
    return counts_df

## Load Vehicle Detection Data

In [ ]:
dfs = []

vehicle_count_paths = get_files_in_directory(VEHICLE_COUNTS_ROOT_PATH)

for vehicle_count_path in vehicle_count_paths:
  df = pd.read_csv(vehicle_count_path)

  df.name = df.iloc[0]['image_id']
  print(df.name)
  dfs.append(df)

dfs[1].head()

blackburn_30361032
luton_m1_2557a
trafford_m60_9083a
havering_m25_5790a
hounslow_m4_2188b
havering_m25_5790b
trafford_m60_9086b
luton_m1_2557b
blackburn_30361033
hounslow_m4_2188a


,image_id,x_min,x_max,y_min,y_max,category_name,area
0,luton_m1_2557a,583.731293,599.885468,1152.652158,1167.770770,Small Car,244
1,luton_m1_2557a,95.104782,108.473045,194.094879,208.805756,Small Car,196
2,luton_m1_2557a,564.606995,582.142776,1088.034058,1105.921570,Small Car,313
3,luton_m1_2557a,102.779465,112.611900,174.205750,189.725266,Small Car,152
4,luton_m1_2557a,180.295212,188.158371,328.243683,340.970886,Small Car,100


## Load Emissions Data

In [ ]:
df_emissions_factors = pd.read_csv(EMISSIONS_FACTORS_PATH)

df_emissions_factors['Total'] = df_emissions_factors.sum(axis=1)

df_emissions_factors

<ipython-input-31-77218a245191>:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_emissions_factors['Total'] = df_emissions_factors.sum(axis=1)


,Vehicle Type,NOx,PM10,PM2.5,CO,VOC,NH3,SO2,Benzene,N2O,Total
0,Petrol cars,0.065000,0.001000,0.001000,0.5830,0.090000,0.013000,0.00000,0.002000,0.001000,0.756000
1,Diesel cars,0.517000,0.008000,0.008000,0.0470,0.004000,0.004000,0.00100,0.000000,0.006000,0.595000
2,Petrol LGVs,0.094000,0.001000,0.001000,1.2340,0.085000,0.017000,0.00100,0.002000,0.002000,1.437000
3,Diesel LGVs,0.808000,0.007000,0.007000,0.0580,0.008000,0.005000,0.00100,0.000000,0.006000,0.900000
4,Rigid HGVs,1.428000,0.020000,0.020000,0.4380,0.039000,0.009000,0.00200,0.000000,0.031000,1.987000
5,Artic HGVs,0.609000,0.011000,0.011000,0.2950,0.027000,0.009000,0.00300,0.000000,0.052000,1.017000
6,Buses,2.602000,0.030000,0.030000,0.7530,0.056000,0.008000,0.00300,0.000000,0.032000,3.514000
7,M/cycle,0.078934,0.007047,0.007047,2.3038,0.280368,0.001973,0.00036,0.012611,0.001822,2.693961


## Convert Vehicle Detection into Compatible Emissions Categories

In [ ]:
dfs = convert_category_names(dfs, XVIEW_TO_UK_MAPPING)

dfs[0].head()

,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type
0,blackburn_30361032,203.576706,225.487869,1076.689453,1092.814453,Small Car,353,cars_and_taxis
1,blackburn_30361032,1975.495361,1990.630280,521.321564,538.422928,Small Car,258,cars_and_taxis
2,blackburn_30361032,2237.856262,2248.048431,209.296661,219.918777,Small Car,108,cars_and_taxis
3,blackburn_30361032,1067.385925,1079.589386,1027.310608,1036.435486,Small Car,111,cars_and_taxis
4,blackburn_30361032,1979.897575,1991.545563,514.580276,524.567917,Small Car,116,cars_and_taxis


## AADT by Number of Vehicles

### Load AADT data

In [ ]:
aadt_paths = get_files_in_directory(AADT_ROOT_PATH)

for aadt_path in aadt_paths:
  df_aadt = pd.read_csv(aadt_path, sep = ',', skipinitialspace = True)

  for df in dfs:
    if df.iloc[0]['image_id'] == df_aadt.iloc[0]['image_id']:
      df['aadt'] = df_aadt.iloc[0]['aadt']

dfs[0].head()

,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type,aadt
0,blackburn_30361032,203.576706,225.487869,1076.689453,1092.814453,Small Car,353,cars_and_taxis,39665.24
1,blackburn_30361032,1975.495361,1990.630280,521.321564,538.422928,Small Car,258,cars_and_taxis,39665.24
2,blackburn_30361032,2237.856262,2248.048431,209.296661,219.918777,Small Car,108,cars_and_taxis,39665.24
3,blackburn_30361032,1067.385925,1079.589386,1027.310608,1036.435486,Small Car,111,cars_and_taxis,39665.24
4,blackburn_30361032,1979.897575,1991.545563,514.580276,524.567917,Small Car,116,cars_and_taxis,39665.24


## Ratio of AADT to each Vehicle Category

In [ ]:
for df in dfs:

  reciprocal_vehicle_count = 1 / len(df)

  if 'aadt' in df:
    aadt = df.iloc[0]['aadt']

    print("image_id: {}, aadt: {}".format(df.iloc[0]['image_id'], aadt))
    df['aadt_vehicles'] = aadt * reciprocal_vehicle_count

dfs[0].head()

image_id: blackburn_30361032, aadt: 39665.24
image_id: luton_m1_2557a, aadt: 73846.7
image_id: trafford_m60_9083a, aadt: 67757.84
image_id: havering_m25_5790a, aadt: 67524.96
image_id: hounslow_m4_2188b, aadt: 84524.03
image_id: havering_m25_5790b, aadt: 69278.06
image_id: trafford_m60_9086b, aadt: 72854.49
image_id: luton_m1_2557b, aadt: 75243.8
image_id: blackburn_30361033, aadt: 39155.9
image_id: hounslow_m4_2188a, aadt: 74373.16


,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type,aadt,aadt_vehicles
0,blackburn_30361032,203.576706,225.487869,1076.689453,1092.814453,Small Car,353,cars_and_taxis,39665.24,1043.822105
1,blackburn_30361032,1975.495361,1990.630280,521.321564,538.422928,Small Car,258,cars_and_taxis,39665.24,1043.822105
2,blackburn_30361032,2237.856262,2248.048431,209.296661,219.918777,Small Car,108,cars_and_taxis,39665.24,1043.822105
3,blackburn_30361032,1067.385925,1079.589386,1027.310608,1036.435486,Small Car,111,cars_and_taxis,39665.24,1043.822105
4,blackburn_30361032,1979.897575,1991.545563,514.580276,524.567917,Small Car,116,cars_and_taxis,39665.24,1043.822105


In [ ]:
dfs[4]

,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type,aadt,aadt_vehicles
0,hounslow_m4_2188b,399.571594,428.061829,1369.149994,1389.839020,Small Car,589,cars_and_taxis,84524.03,982.837558
1,hounslow_m4_2188b,2552.221313,2581.310120,729.291046,750.035583,Small Car,603,cars_and_taxis,84524.03,982.837558
2,hounslow_m4_2188b,874.834702,894.299416,1241.165619,1256.303314,Small Car,294,cars_and_taxis,84524.03,982.837558
3,hounslow_m4_2188b,1179.386902,1200.746979,1163.879715,1179.734222,Small Car,338,cars_and_taxis,84524.03,982.837558
4,hounslow_m4_2188b,929.827423,965.741592,1226.366455,1239.061951,Small Car,455,cars_and_taxis,84524.03,982.837558
...,...,...,...,...,...,...,...,...,...,...
81,hounslow_m4_2188b,1247.802002,1293.842377,1125.134003,1150.947540,Trailer,1188,lgvs,84524.03,982.837558
82,hounslow_m4_2188b,1634.611870,1664.000000,1046.791931,1064.490402,Trailer,520,lgvs,84524.03,982.837558
83,hounslow_m4_2188b,1742.258423,1781.474243,1020.829132,1039.544403,Trailer,733,lgvs,84524.03,982.837558
84,hounslow_m4_2188b,289.879181,329.395874,1395.783112,1411.584961,Trailer,624,lgvs,84524.03,982.837558


## Calculate GHG Emissions 


**TODO: Add mapping for mpg of each vehicle type to calculation for improved accuracy**

In [ ]:
aadt_emissions = []

for df in dfs:
  la_name_id = df.iloc[0]['image_id']
  ghg_emissions = 0
  LENGTH = 0
  litres = 0

  print(la_name_id)

  if 'aadt' in df:

    aadt = df.iloc[0]['aadt']

    if la_name_id.find('blackburn') != -1:
      LENGTH = BLACKBURN_ROAD_LENGTH

    elif la_name_id.find('luton') != -1:
      LENGTH = LUTON_ROAD_LENGTH

    elif la_name_id.find('hounslow') != -1:
      LENGTH = HOUNSLOW_ROAD_LENGTH

    elif la_name_id.find('havering') != -1:
      LENGTH = HAVERING_ROAD_LENGTH

    elif la_name_id.find('trafford') != -1:
      LENGTH = 9.98

    for i in range(len(df)):

      vehicle_km_travel = LENGTH * df.iloc[i]['aadt_vehicles'] * 365

      vehicle_km_litre = VEHICLE_KM_PER_LITRE_MAPPING[df.iloc[i]['Vehicle Type']]

      litres += vehicle_km_travel / vehicle_km_litre # litres

    emissions = litres * PETROL_DIESEL_AVERAGE

    ghg_emissions += np.round(emissions, 1)

    ghg_emissions = ghg_emissions * KG_TO_KT

    print("LA Count Site: {}, AADT Prediction: {}, GHG Emissions Prediction: {}".format(la_name_id, aadt, ghg_emissions))

    save_float_to_csv(ghg_emissions, 'ghg_emissions', image_id=la_name_id, file_name=GHG_EMISSIONS_ROOT_PATH+'ghg_emissions_'+la_name_id+'.csv')
    aadt_emissions.append((la_name_id, ghg_emissions))

blackburn_30361032
LA Count Site: blackburn_30361032, AADT Prediction: 39665.24, GHG Emissions Prediction: 29.938238899999998
luton_m1_2557a
LA Count Site: luton_m1_2557a, AADT Prediction: 73846.7, GHG Emissions Prediction: 15.648853599999999
trafford_m60_9083a
LA Count Site: trafford_m60_9083a, AADT Prediction: 67757.84, GHG Emissions Prediction: 50.274316799999994
havering_m25_5790a
LA Count Site: havering_m25_5790a, AADT Prediction: 67524.96, GHG Emissions Prediction: 105.7059814
hounslow_m4_2188b
LA Count Site: hounslow_m4_2188b, AADT Prediction: 84524.03, GHG Emissions Prediction: 61.04412129999999
havering_m25_5790b
LA Count Site: havering_m25_5790b, AADT Prediction: 69278.06, GHG Emissions Prediction: 73.9294644
trafford_m60_9086b
LA Count Site: trafford_m60_9086b, AADT Prediction: 72854.49, GHG Emissions Prediction: 38.5180067
luton_m1_2557b
LA Count Site: luton_m1_2557b, AADT Prediction: 75243.8, GHG Emissions Prediction: 19.084711199999997
blackburn_30361033
LA Count Site: bl